In [2]:
!pip install -r requirements.txt

  Found existing installation: tqdm 4.38.0
    Uninstalling tqdm-4.38.0:
      Successfully uninstalled tqdm-4.38.0


In [1]:
import numpy as np 
import pandas as pd 
from cryptography.fernet import Fernet
from datetime import datetime, date, timedelta
from tqdm import tqdm_notebook as tqdm
import os
import pymssql as mssql
import calendar
import time
from multiprocessing import Pool
from functools import partial

import ALL_STOCK_preprocess_function as func
import VWAP_feature_function_rolling_week as func1

In [2]:
# part = '1'
# d1 = date(2019, 10, 1)
# d2 = date(2019, 10, 15)

In [10]:
# part = '2'
d1 = date(2019, 10, 11)
d2 = date(2019, 10, 31)

In [2]:
# part = '3'
# d1 = date(2019, 10, 13)
# d2 = date(2019, 10, 14)

In [5]:
# part = '4'
# d1 = date(2019, 10, 19)
# d2 = date(2019, 10, 24)

In [ ]:
# part = '5'
# d1 = date(2019, 10, 25)
# d2 = date(2019, 10, 31)

In [11]:
delta = d2 - d1         # timedelta
date_list = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
total_list = []


for _, end_date in enumerate(tqdm(date_list)):
    if end_date.weekday() in [5, 6]:
        continue
    
    d = func.stock_query(end_date)
    stock, index, industry_index = d[0], d[1], d[2]
    df_list = [group[1] for group in stock.groupby(stock['StockNo'])]
    timedf = pd.DataFrame(d[0]['ts'].unique(), columns=['ts'])
    timedf = timedf.sort_values(by='ts')
    output_list = []

    if __name__ == '__main__':
        with Pool(processes=5) as pool:
            for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func.FillMissingTime, timedf=timedf, end_date=end_date), df_list), total=len(df_list)), 1):
                    if x[0]:
                        output_list.append(x[1])
                    else:
                        continue
    
    
    if len(output_list) == 0:
        continue
        
    df = pd.concat(output_list)
    
    df_list = [group[1] for group in df.groupby(df['StockNo'])]
    output_list = []

    if __name__ == '__main__':
        with Pool(processes=5) as pool:
            for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func.merge_index, index=index, industry_index=industry_index), df_list), total=len(df_list)), 1):
                    output_list.append(x)

    df = pd.concat(output_list)
    
    columns_dict = {'lag': ['index_close', 'industry_close', 'close', 'VWAP', 'VWAP_day5'],
               'return': ['index', 'industry', 'close', 'VWAP', 'VWAP_day5'],
               'ratio': ['index_close', 'index_vol', 'index_return', 'industry_close', 'industry_vol', 'industry_return', 'VWAP_return', 'close_return', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
               'momentum': ['index_close', 'index_vol', 'index_return', 'industry_close', 'industry_vol', 'industry_return', 'VWAP_return', 'close_return', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
                'moment': ['index_close', 'index_vol', 'industry_close', 'industry_vol', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
                'buyer': ['foreign_buy', 'investment_buy', 'dealer_buy']
               }
    
    df_list = [group[1] for group in df.groupby(df['StockNo'])]
    output_list = []

    if __name__ == '__main__':
        with Pool(processes=5) as pool:
            for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func1.get_technical_indicators, columns_dict=columns_dict), df_list), total=len(df_list)), 1):
                output_list.append(x)

    
    df = pd.concat(output_list, axis=0)    
    df_last = df[df.ts.dt.date == end_date]

    total_list.append(df_last)

In [12]:
total_df = pd.concat(total_list)

In [13]:
feature_list = ['total', 'capital', 'index_open', 'index_high', 'index_low', 'index_close',
       'index_return', 'index_vol', 'industry_return',
       'foreign_ratio',
       'investment_ratio', 'corporation_ratio', 'close_return', 'VWAP_return',
       'index_VWAP_slope', 'VWAP_day5_return', 'VWAP_day5_return_5',
       'industry_VWAP_slope', 'index_close_slope', 'industry_close_slope',
       'index_close_ratio_day5', 'index_vol_ratio_day5',
       'index_return_ratio_day5', 'industry_close_ratio_day5',
       'industry_vol_ratio_day5', 'industry_return_ratio_day5',
       'VWAP_return_ratio_day5', 'close_return_ratio_day5', 'vol_ratio_day5',
       'VWAP_ratio_day5', 'index_close_ratio_day10', 'index_vol_ratio_day10',
       'index_return_ratio_day10', 'industry_close_ratio_day10',
       'industry_vol_ratio_day10', 'industry_return_ratio_day10',
       'VWAP_return_ratio_day10', 'close_return_ratio_day10',
       'vol_ratio_day10', 'VWAP_ratio_day10', 'index_close_ratio_day15',
       'index_vol_ratio_day15', 'index_return_ratio_day15',
       'industry_close_ratio_day15', 'industry_vol_ratio_day15',
       'industry_return_ratio_day15', 'VWAP_return_ratio_day15',
       'close_return_ratio_day15', 'vol_ratio_day15', 'VWAP_ratio_day15',
       'index_close_momentum_day1', 'index_vol_momentum_day1',
       'index_return_momentum_day1', 'industry_close_momentum_day1',
       'industry_vol_momentum_day1', 'industry_return_momentum_day1',
       'VWAP_return_momentum_day1', 'close_return_momentum_day1',
       'vol_momentum_day1', 'VWAP_momentum_day1', 'index_close_momentum_day2',
       'index_vol_momentum_day2', 'index_return_momentum_day2',
       'industry_close_momentum_day2', 'industry_vol_momentum_day2',
       'industry_return_momentum_day2', 'VWAP_return_momentum_day2',
       'close_return_momentum_day2', 'vol_momentum_day2', 'VWAP_momentum_day2',
       'index_close_momentum_day3', 'index_vol_momentum_day3',
       'index_return_momentum_day3', 'industry_close_momentum_day3',
       'industry_vol_momentum_day3', 'industry_return_momentum_day3',
       'VWAP_return_momentum_day3', 'close_return_momentum_day3',
       'vol_momentum_day3', 'VWAP_momentum_day3',
       'index_close_momentum_day4', 'index_vol_momentum_day4',
       'index_return_momentum_day4', 'industry_close_momentum_day4',
       'industry_vol_momentum_day4', 'industry_return_momentum_day4',
       'VWAP_return_momentum_day4', 'close_return_momentum_day4',
       'vol_momentum_day4', 'VWAP_momentum_day4', 'index_close_momentum_day5',
       'index_vol_momentum_day5', 'index_return_momentum_day5',
       'industry_close_momentum_day5', 'industry_vol_momentum_day5',
       'industry_return_momentum_day5', 'VWAP_return_momentum_day5',
       'close_return_momentum_day5', 'vol_momentum_day5', 'VWAP_momentum_day5',
       'index_close_momentum_day10', 'index_vol_momentum_day10',
       'index_return_momentum_day10', 'industry_close_momentum_day10',
       'industry_vol_momentum_day10', 'industry_return_momentum_day10',
       'VWAP_return_momentum_day10', 'close_return_momentum_day10',
       'vol_momentum_day10', 'VWAP_momentum_day10', 'index_close_momentum_day15',
       'index_vol_momentum_day15', 'index_return_momentum_day15',
       'industry_close_momentum_day15', 'industry_vol_momentum_day15',
       'industry_return_momentum_day15', 'VWAP_return_momentum_day15',
       'close_return_momentum_day15', 'vol_momentum_day15',
       'VWAP_momentum_day15', 'vol_VWAP_corr', 'vol_volatile_corr',
       'index_vol_close_corr', 'index_vol_volatile_corr',
       'industry_vol_close_corr', 'industry_vol_volatile_corr',
       'index_close_skew', 'index_close_kurtosis', 'index_vol_skew',
       'index_vol_kurtosis', 'industry_close_skew', 'industry_close_kurtosis', 'industry_vol_skew',
       'industry_vol_kurtosis', 'vol_skew', 'vol_kurtosis', 'VWAP_skew',
       'VWAP_kurtosis', 'VWAP_day5_skew', 'VWAP_day5_kurtosis',
       'foreign_buy_ratio', 'foreign_buy_day3_ratio', 'foreign_buy_week_ratio',
       'foreign_buy_2week_ratio', 'foreign_buy_3week_ratio',
       'foreign_buy_continuous', 'foreign_buy_buy_indicator',
       'foreign_buy_sell_indicator', 'investment_buy_ratio',
       'investment_buy_day3_ratio', 'investment_buy_week_ratio',
       'investment_buy_2week_ratio', 'investment_buy_3week_ratio',
       'investment_buy_continuous', 'investment_buy_buy_day_ratio',
       'investment_buy_sell_day_ratio', 'dealer_buy_ratio',
       'dealer_buy_day3_ratio', 'dealer_buy_week_ratio',
       'dealer_buy_2week_ratio', 'dealer_buy_3week_ratio',
       'dealer_buy_continuous', 'dealer_buy_buy_day_ratio',
       'dealer_buy_sell_day_ratio', 'index_pvt', 'industry_pvt', 'foreign_pvt',
       'investment_pvt', 'dealer_pvt', 'pvt', 'RSI_15', 'SOk5_1', 'SOd3',
       'SOk15_5', 'SOd5', 'industry', 'ts', 'StockNo']

In [14]:
total_df[feature_list].isna().values.any()

False

In [17]:
total_df[feature_list].to_csv(f'testing_all_201910_p2.csv',index=False)

In [16]:
total_df['ts'].min()

Timestamp('2019-10-14 00:00:00')

In [9]:
len(feature_list)

178

In [18]:
d = func.stock_query(end_date)

In [19]:
d[0]['PctRank']

0        None
1        None
2        None
3        None
4        None
         ... 
95633    None
95634    None
95635    None
95636    None
95637    None
Name: PctRank, Length: 95638, dtype: object